# ONNX2TRT + Yolov5s

# Prerequisites

1. In a terminal session on this Jupyter notebook server, run `aws configure`. This allows this notebook server to access Panorama resources and deploy applications on your behalf.

2. This notebook works best in the Test Utility. Please see [here](https://github.com/aws-samples/aws-panorama-samples/blob/main/docs/EnvironmentSetup.md) to set it up 

3. **VERY IMPORTANT** : This example will build the engine file with dynamic batch size ranges from 1 to 8. If you decide to use batch size larger than 4, please use ATLEAST 2 CAMERAS for this example.

4. **PLEASE READ THE [README](README.md) INCLUDE WITH THIS BEFORE YOU START USING THIS NOTEBOOK**

# Set up

Import libraries for use with this notebook environment, you do not need these libraries when you write your application code.

## Notebook parameters
Global constants that help the notebook create Panorama resources on your behalf.

In [1]:
import boto3
# application name
app_name = 'onnx2trt_app'

## package names and node names
code_package_name = 'onnx2trt_app'
camera_node_name = 'abstract_rtsp_media_source'

# AWS account ID
account_id = boto3.client("sts").get_caller_identity()["Account"]

env: AWS_PROFILE=ppe


## Set up application

Every application uses the creator's AWS Account ID as the prefix to uniquely identifies the application resources. Running `panorama-cli import-application` replaces the generic account Id with your account Id.

In [2]:
!cd ./{app_name} && panorama-cli import-application

Sucessfully imported application


### Build Docker image from Dockerfile

Open a terminal, go to ./dependencies/docker and run:

`sudo docker build -t  trtpt36:latest .`

This step takes a long time (approximately 5hrs.)

### Build app with container

In [3]:
container_asset_name = 'onnx2trt_app'

In [4]:
%%capture captured_output

# Building container image. This process takes time (5min ~ 10min)
# FIXME : without %%capture, browser tab crashes because of too much output from the command.

!cd ./{app_name} && panorama-cli build \
    --container-asset-name {container_asset_name} \
    --package-path packages/{account_id}-{code_package_name}-1.0



In [5]:
stdout_lines = captured_output.stdout.splitlines()
stderr_lines = captured_output.stderr.splitlines()
print("     :")
print("     :")
for line in stdout_lines[-30:] + stderr_lines[-30:]:
    print(line)

     :
     :
docker export --output=onnx2trt_app.tar $(docker create onnx2trt_app:latest)
gzip -1 onnx2trt_app.tar
Updating an existing asset with the same name
Deleting old asset 85d25f6755ad310982d145cd96c367e32067aa7f18e474259b0a7dc7d62474b4.tar.gz
Deleting old descriptor 63a87b807a1a19dea23660a2e2ca168a9377b6dca80fb1bfb3e9e123702f26a8.json
{
    "name": "onnx2trt_app",
    "implementations": [
        {
            "type": "container",
            "assetUri": "c402506ba1175dc1aeca11421f5b0642b3d9bd55f0adae41faec9169e61b29f7.tar.gz",
            "descriptorUri": "081b9484578f179470669c01c800b769b0e5c1a51141761c1ac8a33b514b2998.json",
            "requirements": [
                {
                    "type": "hardware_access",
                    "inferenceAccelerators": [
                        {
                            "deviceType": "nvhost_gpu",
                            "sharedResourcePolicy": {
                                "policy": "allow_all"
                      

### Special flags in package.json

* Step 1 : Before you deploy the application, open TF37_opengpu/onnx_37_app/packages/(account-id)-tf2_4_trt_app-1.0/package.json
* Step 2 : Add the following flags to the package.json

```
"requirements": 
            [{
                    "type" : "hardware_access",
                    "inferenceAccelerators": [ 
                        {
                            "deviceType": "nvhost_gpu",
                            "sharedResourcePolicy": {
                                "policy" : "allow_all"
                            }
                        }
                    ]
            }]
```

The assets should look something like this

```
"assets": [
    {
        "name": "onnx2trt_app",
        "implementations": [
            {
                "type": "container",
                "assetUri": "9a49a98784f4571adacc417f00942dac7ef2e34686eef21dca9fcb7f4b7ffd70.tar.gz",
                "descriptorUri": "4bab130ec48eea84e072d9fe813b947e9d9610b2924099036b0165026a91d306.json",
                "requirements": 
                [{
                    "type" : "hardware_access",
                    "inferenceAccelerators": [ 
                        {
                            "deviceType": "nvhost_gpu",
                            "sharedResourcePolicy": {
                                "policy" : "allow_all"
                            }
                        }
                    ]
                }]
            }
        ]
    }
],
```

### Upload application to Panorama for deploying to devices

In [6]:
# This step takes some time, depending on your network environment.
!cd ./{app_name} && pwd && panorama-cli package-application

/home/ubuntu/workspace/aws-panorama-samples/samples/ONNX2TRT_opengpu/onnx2trt_app
Uploading package onnx2trt_app
Package Version 1.0 is not yet registered, preparing upload
upload: assets/c402506ba1175dc1aeca11421f5b0642b3d9bd55f0adae41faec9169e61b29f7.tar.gz to s3://arn:aws:s3:ap-southeast-1:899004849941:accesspoint/panorama-201125699002-ilamkyisaigd7qma7drecxph5q/201125699002/nodePackages/onnx2trt_app/binaries/c402506ba1175dc1aeca11421f5b0642b3d9bd55f0adae41faec9169e61b29f7.tar.gz
upload: assets/081b9484578f179470669c01c800b769b0e5c1a51141761c1ac8a33b514b2998.json to s3://arn:aws:s3:ap-southeast-1:899004849941:accesspoint/panorama-201125699002-ilamkyisaigd7qma7drecxph5q/201125699002/nodePackages/onnx2trt_app/binaries/081b9484578f179470669c01c800b769b0e5c1a51141761c1ac8a33b514b2998.json
{
    "ETag": "\"66a949e3518ecb158196e5e1d924991f\"",
    "ServerSideEncryption": "AES256",
    "VersionId": "mkRF4CzRGbnJm2hk3aztNvLxG0_ADsm9"
}
Called register package version for onnx2trt_app with p

### Ready for deploying to a device

Congrats! Your app is now ready to deploy to a device. Next, you can continue in this notebook to deploy the app programmatically or you can go to the Panorama console and deploying using the AWS Console. The console makes it easier to select camera streams and select the devices you want to deploy to. Programmatic deployment is faster to complete and easier to automate.

# Deploy app to device

* Step 1: Copy the contents of./graphs/onnx2trt_app/graph.json
* Step 2 : Go to the AWS Panorama Console, click Deploy Application
* Step 3 : Paste the contents you just copied
* Step 4 : Select the device
* Step 5: Select atleast 2 cameras if batch size > 4
* Step 6: Configure the batch size with desired batch size.
* Step 7 : Deploy

The deployment should take about 30 minutes, after deployment it will takes 12 ~ 20 mins for engine build. After build, the engine will be cached so that rebooting the device does not require another build.